`ProPublica Recidivism/COMPAS` :<br>
    The COMPAS dataset (Correctional Offender Management Profiling for Alternative Sanctions) is a dataset used for binary classification tasks in the field of criminal justice. The goal is to predict whether a defendant will re-offend within two years of release, based on various demographic and criminal history features.<br>
    Reference Link: https://www.propublica.org/datastore/dataset/compas-recidivism-risk-score-data-and-analysis

In [1]:
import pandas as pd
import numpy as np

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)\

import facts
from facts.clean import clean_dataset
from facts import valid_ifthens_with_coverage_correctness, rules2rulesbyif
from facts.models import customLogisticRegression
from facts.parameters import ParameterProxy
from facts.formatting import recourse_report_reverse, print_recourse_report, print_recourse_report_cumulative, print_recourse_report_KStest_cumulative
from facts.utils import load_rules_by_if
from aif360.sklearn.datasets import fetch_compas
from facts.fairness_metrics_aggr import get_diff_table, get_comb_df, get_analysis_dfs, get_diff_real_diff_drop,get_other_ranks_divided,get_map_metric_to_max_rank
import warnings
warnings.filterwarnings('ignore')

In [2]:
cost_budget = 10
top_count = 20
c_inf = 5

In [3]:
X, y = fetch_compas()
X['target'] = y.values

sensitive_attribute = "race"
df = clean_dataset(X,'compas')
y = df['target']
X = df.drop('target', axis=1)

X.head()

,sex,age_cat,race,juv_fel_count,juv_misd_count,juv_other_count,priors_count,c_charge_degree
0,Male,25 - 45,African-American,0,0,0,"(-0.1, 1.0]",F
1,Male,10-25,African-American,0,0,1,"(1.0, 5.0]",F
2,Male,25 - 45,Caucasian,0,0,0,"(10.0, 15.0]",F
3,Female,25 - 45,Caucasian,0,0,0,"(-0.1, 1.0]",M
4,Male,25 - 45,Caucasian,0,0,0,"(-0.1, 1.0]",F


In [4]:
rules_with_atomic_correctness = load_rules_by_if("rulesCompas_atomic.data")
rules_with_cumulative_correctness = load_rules_by_if("rulesCompas_cumulative.data")

In [5]:
num_features = X._get_numeric_data().columns.to_list()
cate_features = X.select_dtypes(include=['object','category']).columns.to_list()
ord_features = []
feature_weights = {'sex':100,	'age_cat':10,	'race':100,	'juv_fel_count':1,	'juv_fel_count':1,	'juv_other_count':1,	'priors_count':1,	'c_charge_degree':1}

features_with_binary_cost = cate_features
features_with_proportional_cost = num_features


comparators = facts.feature_change_builder(
    X,
    num_cols=features_with_proportional_cost,
    cate_cols=features_with_binary_cost,
    ord_cols=ord_features,
    feature_weights=feature_weights,
    num_normalization=False,
    #feats_to_normalize = ["capital-gain","capital-loss"]
)
params = ParameterProxy(featureChanges=comparators)
facts.update_costs_cumulative(rules_with_cumulative_correctness, params)

# Macro Viewpoint

## Equal Cost of Effectiveness (Macro)

In [6]:
top_rules, subgroup_costs = facts.select_rules_subset(
    rules_with_atomic_correctness,
    metric="min-above-thr",
    sort_strategy="generic-sorting-ignore-exists-subgroup-empty",
    top_count=50,
    cor_threshold=0.7,
    filter_sequence = [
        "remove-contained",
        "remove-below-thr",
        # "remove-fair-rules",
        "keep-only-min-change"
    ],
    params=params
)

print_recourse_report(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True,
    metric_name = 'Equal Cost of Effectiveness (threshold = 0.7)'
)

If age_cat = 10-25, c_charge_degree = F, juv_fel_count = 0, juv_other_count = 0, sex = Male:
	Protected Subgroup 'Caucasian', 21.54% covered
		Make c_charge_degree = M with effectiveness 71.43%.
		Aggregate cost of the above recourses = 1.00
	Protected Subgroup 'African-American', 20.89% covered
		Make age_cat = Greater than 45 with effectiveness 90.74%.
		Aggregate cost of the above recourses = 10.00
	Bias against African-American due to Equal Cost of Effectiveness (threshold = 0.7). Unfairness score = 9.
If age_cat = 10-25, c_charge_degree = F, juv_misd_count = 0, juv_other_count = 0:
	Protected Subgroup 'Caucasian', 21.03% covered
		Make c_charge_degree = M with effectiveness 73.17%.
		Aggregate cost of the above recourses = 1.00
	Protected Subgroup 'African-American', 20.12% covered
		Make age_cat = 25 - 45 with effectiveness 73.08%.
		Make age_cat = Greater than 45 with effectiveness 93.27%.
		Aggregate cost of the above recourses = 10.00
	Bias against African-American due to Equa

## Equal Choice for Recourse(Macro)

In [7]:
top_rules, subgroup_costs = facts.select_rules_subset(
    rules_with_atomic_correctness,
    metric="num-above-thr",
    sort_strategy="generic-sorting-ignore-forall-subgroups-empty",
    top_count=50,
    cor_threshold=0.7,
    filter_sequence = [
        "remove-contained",
        "remove-below-thr",
        # "remove-fair-rules"
    ],
    params=params
)

print_recourse_report(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True,
    metric_name = 'Equal Choice for Recourse (threshold = 0.7)'
)

If age_cat = 25 - 45, c_charge_degree = F, juv_other_count = 1:
	Protected Subgroup 'Caucasian', 2.05% covered
		Make c_charge_degree = M, juv_other_count = 0 with effectiveness 75.00%.
		Make age_cat = Greater than 45, juv_other_count = 0 with effectiveness 75.00%.
		Make age_cat = Greater than 45, c_charge_degree = M, juv_other_count = 0 with effectiveness 75.00%.
		Aggregate cost of the above recourses = -3.00
	Protected Subgroup 'African-American', 3.87% covered
		No recourses for this subgroup!
		Aggregate cost of the above recourses = 0.00
	Bias against African-American due to Equal Choice for Recourse (threshold = 0.7). Unfairness score = 3.
If age_cat = 10-25, c_charge_degree = F, sex = Female:
	Protected Subgroup 'Caucasian', 1.54% covered
		Make age_cat = Greater than 45, c_charge_degree = M with effectiveness 100.00%.
		Aggregate cost of the above recourses = -1.00
	Protected Subgroup 'African-American', 1.55% covered
		Make age_cat = 25 - 45 with effectiveness 87.50%.
		Mak

# Micro Viewpoint

## Equal Conditional Mean Recourse

In [8]:
top_rules, subgroup_costs = facts.select_rules_subset_cumulative(
    rules_with_cumulative_correctness,
    metric="fairness-of-mean-recourse-conditional",
    sort_strategy="generic-sorting-ignore-exists-subgroup-empty",
    top_count=top_count,
    filter_sequence = [
        "remove-contained",
        "remove-fair-rules"
    ],
    params=params
)

print_recourse_report_cumulative(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True,
    show_then_costs=True,
    metric_name = 'Equal Conditional Mean Recourse'
)

If age_cat = 25 - 45, juv_other_count = 2:
	Protected Subgroup 'Caucasian', 2.05% covered
		Make juv_other_count = 0 with effectiveness 50.00% and counterfactual cost = 2.
		Make age_cat = Greater than 45, juv_other_count = 0 with effectiveness 50.00% and counterfactual cost = 12.
		Aggregate cost of the above recourses = 2.00
	Protected Subgroup 'African-American', 1.55% covered
		Make juv_other_count = 0 with effectiveness 12.50% and counterfactual cost = 2.
		Make age_cat = Greater than 45, juv_other_count = 0 with effectiveness 50.00% and counterfactual cost = 12.
		Aggregate cost of the above recourses = 9.50
	Bias against African-American due to Equal Conditional Mean Recourse. Unfairness score = 7.5.
If age_cat = 25 - 45, juv_misd_count = 0, juv_other_count = 1, sex = Male:
	Protected Subgroup 'Caucasian', 1.54% covered
		Make juv_other_count = 0 with effectiveness 0.00% and counterfactual cost = 1.
		Make age_cat = Greater than 45, juv_other_count = 0 with effectiveness 66.67% 

## Equal Cost of Effectiveness (Micro)

In [9]:
top_rules, subgroup_costs = facts.select_rules_subset_cumulative(
    rules_with_cumulative_correctness,
    metric="min-above-corr",
    sort_strategy="generic-sorting-ignore-exists-subgroup-empty",
    top_count=top_count,
    cor_threshold = 0.7,
    filter_sequence = [
        "remove-contained",
        "remove-fair-rules",
        "keep-cheap-rules-above-thr-cor",
        
        
    ],
    params=params
)

print_recourse_report_cumulative(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True,
    show_then_costs=True,
    metric_name = 'Equal Cost of Effectiveness (threshold = 0.7)'
)

If age_cat = 10-25, c_charge_degree = F, juv_fel_count = 0, juv_other_count = 0, sex = Male:
	Protected Subgroup 'Caucasian', 21.54% covered
		Make c_charge_degree = M with effectiveness 71.43% and counterfactual cost = 1.
		Aggregate cost of the above recourses = 1.00
	Protected Subgroup 'African-American', 20.89% covered
		Make c_charge_degree = M with effectiveness 67.59% and counterfactual cost = 1.
		Make age_cat = 25 - 45 with effectiveness 67.59% and counterfactual cost = 10.
		Make age_cat = Greater than 45 with effectiveness 90.74% and counterfactual cost = 10.
		Aggregate cost of the above recourses = 10.00
	Bias against African-American due to Equal Cost of Effectiveness (threshold = 0.7). Unfairness score = 9.
If age_cat = 10-25, c_charge_degree = F, juv_misd_count = 0, juv_other_count = 0:
	Protected Subgroup 'Caucasian', 21.03% covered
		Make c_charge_degree = M with effectiveness 73.17% and counterfactual cost = 1.
		Aggregate cost of the above recourses = 1.00
	Protecte

## Equal Effectiveness

In [10]:
top_rules, subgroup_costs = facts.select_rules_subset_cumulative(
    rules_with_cumulative_correctness,
    metric="total-correctness",
    sort_strategy="generic-sorting-ignore-forall-subgroups-empty",
    top_count=top_count,
    filter_sequence = [
        "remove-contained",
        "remove-fair-rules",
    ],
    params=params
)

print_recourse_report_cumulative(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True,
    show_then_costs=True,
    correctness_metric=True,
    metric_name = 'Equal Effectiveness'
)

If juv_other_count = 3:
	Protected Subgroup 'Caucasian', 1.54% covered
		Make juv_other_count = 0 with effectiveness 66.67% and counterfactual cost = 3.
		Aggregate cost of the above recourses = 0.67
	Protected Subgroup 'African-American', 1.16% covered
		Make juv_other_count = 0 with effectiveness 0.00% and counterfactual cost = 3.
		Aggregate cost of the above recourses = 0.00
	Bias against African-American due to Equal Effectiveness. Unfairness score = 0.667.
If age_cat = 10-25, c_charge_degree = F, juv_fel_count = 0, juv_misd_count = 1:
	Protected Subgroup 'Caucasian', 1.03% covered
		Make juv_misd_count = 0 with effectiveness 0.00% and counterfactual cost = 1.
		Make c_charge_degree = M, juv_misd_count = 0 with effectiveness 0.00% and counterfactual cost = 2.
		Make age_cat = 25 - 45, juv_misd_count = 0 with effectiveness 0.00% and counterfactual cost = 11.
		Make age_cat = Greater than 45, juv_misd_count = 0 with effectiveness 50.00% and counterfactual cost = 11.
		Make age_cat =

## Fair Effectiveness-Cost Trade-Off

In [11]:
# taken from the other notebooks
affected_pop_sizes = {"African-American": 483, "Caucasian": 262}

top_rules,unfairness = facts.select_rules_subset_KStest(
    rules_with_cumulative_correctness,
    affected_pop_sizes,
    top_count=top_count
)

print_recourse_report_KStest_cumulative(
    top_rules,
    population_sizes=affected_pop_sizes,
    unfairness = unfairness,
    show_then_costs=True,
    metric_name = 'Fair Effectiveness-Cost Trade-Off'
    # show_cumulative_plots=True
)

If age_cat = 25 - 45, c_charge_degree = F, sex = Male:
	Protected Subgroup 'Caucasian', 43.59% covered out of 262
		Make c_charge_degree = M with effectiveness 54.12% and counterfactual cost = 1.
		Make age_cat = Greater than 45 with effectiveness 57.65% and counterfactual cost = 10.
		Make age_cat = Greater than 45, c_charge_degree = M with effectiveness 70.59% and counterfactual cost = 11.
	Protected Subgroup 'African-American', 44.10% covered out of 483
		Make c_charge_degree = M with effectiveness 31.14% and counterfactual cost = 1.
		Make age_cat = Greater than 45 with effectiveness 35.96% and counterfactual cost = 10.
		Make age_cat = Greater than 45, c_charge_degree = M with effectiveness 38.60% and counterfactual cost = 11.
	 Unfairness based on the Fair Effectiveness-Cost Trade-Off = 2.76.
If age_cat = 25 - 45, c_charge_degree = F:
	Protected Subgroup 'Caucasian', 46.67% covered out of 262
		Make c_charge_degree = M with effectiveness 51.65% and counterfactual cost = 1.
		Make

## Equal Effectiveness within Budget (Micro)

In [12]:
top_rules, subgroup_costs = facts.select_rules_subset_cumulative(
    rules_with_cumulative_correctness,
    metric="max-upto-cost",
    sort_strategy="generic-sorting-ignore-exists-subgroup-empty",
    top_count=top_count,
    cost_threshold = cost_budget,
    filter_sequence = [
        "remove-contained",
        "remove-fair-rules",
        "remove-above-thr-cost"
    ],
    params=params
)

print_recourse_report_cumulative(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True,
    show_then_costs=True,
    correctness_metric = True,
    metric_name = 'Equal Effectiveness within Budget (Micro)'
)

If juv_other_count = 3:
	Protected Subgroup 'Caucasian', 1.54% covered
		Make juv_other_count = 0 with effectiveness 66.67% and counterfactual cost = 3.
		Aggregate cost of the above recourses = 0.67
	Protected Subgroup 'African-American', 1.16% covered
		Make juv_other_count = 0 with effectiveness 0.00% and counterfactual cost = 3.
		Aggregate cost of the above recourses = 0.00
	Bias against African-American due to Equal Effectiveness within Budget (Micro). Unfairness score = 0.667.
If age_cat = 10-25, juv_other_count = 1, priors_count = (1.0, 5.0]:
	Protected Subgroup 'Caucasian', 3.08% covered
		Make juv_other_count = 0, priors_count = (-0.1, 1.0] with effectiveness 66.67% and counterfactual cost = 2.
		Aggregate cost of the above recourses = 0.67
	Protected Subgroup 'African-American', 1.93% covered
		Make juv_other_count = 0, priors_count = (-0.1, 1.0] with effectiveness 20.00% and counterfactual cost = 2.
		Aggregate cost of the above recourses = 0.20
	Bias against African-Americ

## Get ranking of subgroups based on metrics

In [13]:
rules_with_both = {}
for ifc, all_thens in rules_with_cumulative_correctness.items():
    new_all_thens = {}
    for sg, (cov, thens_cum) in all_thens.items():
        thens_atom = rules_with_atomic_correctness[ifc][sg][1]
        thens_atom_dict = {then: atom_cor for then, atom_cor in thens_atom}
        new_all_thens[sg] = (cov, [(then, thens_atom_dict[then], cumcor) for then, cumcor, _cost in thens_cum])
    rules_with_both[ifc] = new_all_thens

In [14]:
from facts.fairness_metrics_aggr import make_table, auto_budget_calculation
budgets = auto_budget_calculation(rules_with_cumulative_correctness, cor_thres=0.5, percentiles=[0.3, 0.6, 0.9])

df = make_table(
    rules_with_both,
    sensitive_attribute_vals=["Caucasian", "African-American"],
    effectiveness_thresholds=[0.3, 0.7],
    cost_budgets=budgets,
    params=params
)

df.drop(['weighted-average', 'mean-cost-cinf'], axis=1, inplace=True)
df = df.loc[:, ~df.columns.duplicated()]


In [15]:
#table of differences between Male and Female for each metric
diff = get_diff_table(df, sensitive_attribute_vals=["Caucasian", "African-American"], with_abs = True)
diff = diff.set_index('subgroup')
ranked = diff.drop(columns=[('Fair Effectiveness-Cost Trade-Off','bias')]).mask(diff == 0).rank(ascending = False,axis=0,method='dense')
ranked = ranked.replace(np.nan,"Fair")

rev_bias_metrics = ['Equal Effectiveness', 'Equal Effectiveness within Budget']
comb_df = get_comb_df(df, ranked, diff, rev_bias_metrics, sensitive_attribute_vals=["Caucasian","African-American"])
diff_real_val, _, = get_diff_real_diff_drop(df, diff, sensitive_attribute_vals=["Caucasian", "African-American"])
analysis_df,rank_analysis_df = get_analysis_dfs(comb_df, diff_real_val, rev_bias_metrics, ranked, sensitive_attribute_vals=["Caucasian", "African-American"], percentage=0.1)

In [16]:
comb_df.head()

(Equal Cost of Effectiveness(Macro), 0.3)  \
                                                                                        rank   
subgroup                                                                                       
c_charge_degree = F                                                                     Fair   
c_charge_degree = F, juv_fel_count = 0                                                  Fair   
c_charge_degree = F, juv_misd_count = 0                                                 Fair   
c_charge_degree = F, sex = Male                                                         Fair   
c_charge_degree = F, juv_fel_count = 0, juv_mis...                                      Fair   

                                                                       \
                                                   score bias against   
subgroup                                                                
c_charge_degree = F                                  0.0         Fair   
c_charge_degree = F, juv_fel_count = 0               0.0         Fair   
c_charge_degree = F, juv_misd_count = 0              0.0         Fair   
c_charge_degree = F, sex = Male                      0.0         Fair   
c_charge_degree = F, juv_fel_count = 0, juv_mis...   0.0         Fair   

                                                   (Equal Cost of Effectiveness(Macro), 0.7)  \
                                                                                        rank   
subgroup                                                                                       
c_charge_degree = F                                                                     Fair   
c_charge_degree = F, juv_fel_count = 0                                                  Fair   
c_charge_degree = F, juv_misd_count = 0                                                 Fair   
c_charge_degree = F, sex = Male                                                         Fair   
c_charge_degree = F, juv_fel_count = 0, juv_mis...                                      Fair   

                                                                       \
                                                   score bias against   
subgroup                                                                
c_charge_degree = F                                  0.0         Fair   
c_charge_degree = F, juv_fel_count = 0               0.0         Fair   
c_charge_degree = F, juv_misd_count = 0              0.0         Fair   
c_charge_degree = F, sex = Male                      0.0         Fair   
c_charge_degree = F, juv_fel_count = 0, juv_mis...   0.0         Fair   

                                                   (Equal Choice for Recourse, 0.3)  \
                                                                               rank   
subgroup                                                                              
c_charge_degree = F                                                            Fair   
c_charge_degree = F, juv_fel_count = 0                                         Fair   
c_charge_degree = F, juv_misd_count = 0                                        Fair   
c_charge_degree = F, sex = Male                                                Fair   
c_charge_degree = F, juv_fel_count = 0, juv_mis...                             Fair   

                                                                       \
                                                   score bias against   
subgroup                                                                
c_charge_degree = F                                  0.0         Fair   
c_charge_degree = F, juv_fel_count = 0               0.0         Fair   
c_charge_degree = F, juv_misd_count = 0              0.0         Fair   
c_charge_degree = F, sex = Male                      0.0         Fair   
c_charge_degree = F, juv_fel_count = 0, juv_mis...   0.0         Fair   

                                                   (Equal Cho

In [17]:
analysis_df

,Rank = 1 Count,Caucasian bias against Count,African-American bias against Count,Top 10% Caucasian bias against Count,Top 10% African-American bias against Count
"(Equal Cost of Effectiveness(Macro), 0.3)",51,37,82,0,11
"(Equal Cost of Effectiveness(Macro), 0.7)",46,19,43,0,6
"(Equal Choice for Recourse, 0.3)",13,108,98,12,8
"(Equal Choice for Recourse, 0.7)",15,31,113,8,6
Equal Effectiveness,1,99,416,14,37
"(Equal Effectiveness within Budget, 1.0)",4,97,365,16,30
"(Equal Effectiveness within Budget, 10.0)",1,117,474,20,39
"(Equal Cost of Effectiveness(Micro), 0.3)",51,37,82,0,11
"(Equal Cost of Effectiveness(Micro), 0.7)",46,19,43,0,6
Equal(Conditional) Mean Recourse,37,130,303,19,24


In [18]:
rank_analysis_df

,"(Equal Cost of Effectiveness(Macro), 0.3)","(Equal Cost of Effectiveness(Macro), 0.7)","(Equal Choice for Recourse, 0.3)","(Equal Choice for Recourse, 0.7)",Equal Effectiveness,"(Equal Effectiveness within Budget, 1.0)","(Equal Effectiveness within Budget, 10.0)","(Equal Cost of Effectiveness(Micro), 0.3)","(Equal Cost of Effectiveness(Micro), 0.7)",Equal(Conditional) Mean Recourse,"(Fair Effectiveness-Cost Trade-Off, value)"
"(Equal Cost of Effectiveness(Macro), 0.3)",1.0,5.0,2.6,4.0,61.7,262.7,157.6,1.0,5.0,239.0,116.8
"(Equal Cost of Effectiveness(Macro), 0.7)",4.8,1.0,3.7,2.5,46.4,169.8,93.3,4.8,1.0,156.0,76.9
"(Equal Choice for Recourse, 0.3)",1.6,3.8,1.0,3.1,30.0,328.0,70.5,1.6,3.8,89.1,60.0
"(Equal Choice for Recourse, 0.7)",4.5,2.3,3.2,1.0,155.3,158.9,22.4,4.5,2.3,77.6,23.1
Equal Effectiveness,1.0,5.0,3.0,4.0,1.0,328.0,1.0,1.0,5.0,1.0,1.0
"(Equal Effectiveness within Budget, 1.0)",4.0,5.0,3.0,3.0,366.0,1.0,396.0,4.0,5.0,124.0,1.0
"(Equal Effectiveness within Budget, 10.0)",1.0,5.0,3.0,4.0,1.0,328.0,1.0,1.0,5.0,1.0,1.0
"(Equal Cost of Effectiveness(Micro), 0.3)",1.0,5.0,2.6,4.0,61.7,262.7,157.6,1.0,5.0,239.0,116.8
"(Equal Cost of Effectiveness(Micro), 0.7)",4.8,1.0,3.7,2.5,46.4,169.8,93.3,4.8,1.0,156.0,76.9
Equal(Conditional) Mean Recourse,4.9,5.0,4.0,4.0,185.6,253.3,202.7,4.9,5.0,1.0,323.4


In [19]:
max_ranks = get_map_metric_to_max_rank(ranked)
rank_divided = get_other_ranks_divided(rank_analysis_df, max_ranks)
rank_divided

,"(Equal Cost of Effectiveness(Macro), 0.3)","(Equal Cost of Effectiveness(Macro), 0.7)","(Equal Choice for Recourse, 0.3)","(Equal Choice for Recourse, 0.7)",Equal Effectiveness,"(Equal Effectiveness within Budget, 1.0)","(Equal Effectiveness within Budget, 10.0)","(Equal Cost of Effectiveness(Micro), 0.3)","(Equal Cost of Effectiveness(Micro), 0.7)",Equal(Conditional) Mean Recourse,"(Fair Effectiveness-Cost Trade-Off, value)"
"(Equal Cost of Effectiveness(Macro), 0.3)",-,1.0,0.65,1.0,0.169,0.801,0.398,0.2,1.0,0.797,0.226
"(Equal Cost of Effectiveness(Macro), 0.7)",0.96,-,0.925,0.625,0.127,0.518,0.236,0.96,0.2,0.52,0.149
"(Equal Choice for Recourse, 0.3)",0.32,0.76,-,0.775,0.082,1.0,0.178,0.32,0.76,0.297,0.116
"(Equal Choice for Recourse, 0.7)",0.9,0.46,0.8,-,0.424,0.484,0.057,0.9,0.46,0.259,0.045
Equal Effectiveness,0.2,1.0,0.75,1.0,-,1.0,0.003,0.2,1.0,0.003,0.002
"(Equal Effectiveness within Budget, 1.0)",0.8,1.0,0.75,0.75,1.0,-,1.0,0.8,1.0,0.413,0.002
"(Equal Effectiveness within Budget, 10.0)",0.2,1.0,0.75,1.0,0.003,1.0,-,0.2,1.0,0.003,0.002
"(Equal Cost of Effectiveness(Micro), 0.3)",0.2,1.0,0.65,1.0,0.169,0.801,0.398,-,1.0,0.797,0.226
"(Equal Cost of Effectiveness(Micro), 0.7)",0.96,0.2,0.925,0.625,0.127,0.518,0.236,0.96,-,0.52,0.149
Equal(Conditional) Mean Recourse,0.98,1.0,1.0,1.0,0.507,0.772,0.512,0.98,1.0,-,0.627
